In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding,Flatten,Dense,Dropout,Bidirectional
from keras.layers import LSTM
from keras import regularizers

Загружаем данные

In [0]:
data_train = pd.read_csv('/content/gdrive/My Drive/data/data_train.csv')
data_test = pd.read_csv('/content/gdrive/My Drive/data/data_test.csv')

Подготавливаем данные

In [0]:
maxlen = 200  # отсечение остатка отзыва после 200 слов
training_samples = 20000  # Обучение на выборке из  20000 образцов
validation_samples = 5000  # Проверка на выборке из 5 000
max_words = 12000  # Рассмотренные только 12000  наиболее часто используемых слов

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data_train.text)
sequence = tokenizer.texts_to_sequences(data_train.text)
data = pad_sequences(sequence, maxlen=maxlen)

x_train = data[:training_samples]
y_train = data_train.target.values[:training_samples]
x_val = data[training_samples:training_samples + validation_samples]
y_val = data_train.target.values[training_samples:
                                 training_samples + validation_samples]

model = Sequential()
model.add(Embedding(12000, 32))
Dropout(0.1)
model.add(LSTM(100))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=8,
                    batch_size=2048, validation_data=(x_val, y_val))


Готовим тестовые данные

In [0]:
sequence = tokenizer.texts_to_sequences(data_test.text)
x_test = pad_sequences(sequence, maxlen=maxlen)
y_test = data_test.target.values

проверям качество

In [0]:
model.evaluate(x_test,y_test)

сохраняем модели

In [0]:
import pickle

# saving
with open('/content/gdrive/My Drive/data/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('/content/gdrive/My Drive/data/model.pickle', 'wb') as handle:
    pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)

# loading
#with open('tokenizer.pickle', 'rb') as handle:
#    tokenizer = pickle.load(handle)